<a href="https://colab.research.google.com/github/jeet1912/florence-2_ablationStudy/blob/main/code/mimic_cxr_ablation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade gcsfs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.6/199.6 kB 9.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
  Attempting uninstall: gcsfs
    Found existing installation: gcsfs 2025.3.0
    Uninstalling gcsfs-2025.3.0:
      Successfully uninstalled gcsfs-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.

In [ ]:
!pip install --upgrade google-cloud-storage

In [ ]:
!pip install iterative-stratification

In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 9.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.7.0
    Uninstalling fsspec-2025.7.0:
      Successfully uninstalled fsspec-2025.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.7.0 requires fsspec==2025.7.0, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nv

In [ ]:
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 107.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninst

In [ ]:
import pandas as pd
import gcsfs
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import shuffle
from collections import defaultdict
import torch
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import io
import os
from transformers import AutoProcessor, AutoModelForCausalLM
from PIL import Image
import torch.optim as optim
import torch.nn as nn
import copy
import itertools
from evaluate import load
import gc
import time
from transformers import AutoModelForCausalLM, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model
from google.colab import drive
from google.colab import auth
from google.cloud import storage

auth.authenticate_user()
drive.mount('/content/drive', force_remount=True)
plt.style.use('default')
sns.set_palette("colorblind")

Mounted at /content/drive


In [ ]:
class CXR(Dataset):
  def __init__(self, dataframe, processor, max_length):
    super().__init__()
    self.dataframe = dataframe.reset_index(drop=True)
    self.processor = processor
    self.max_length = max_length
    self.prompt = "List pathalogical findings for this chest X-ray:"
    self.storage_client = storage.Client(project='ablation-study')
  def __len__(self):
    return len(self.dataframe)

  def _loadImage(self,subject_id, study_id, dicom_id):
    try:
      bucket_name = "mimic-cxr-jpg-2.1.0.physionet.org"
      image_path = f"files/p{subject_id[:2]}/p{subject_id}/s{study_id}/{dicom_id}.jpg"
      bucket = self.storage_client.bucket(bucket_name, user_project='ablation-study')
      blob = bucket.blob(image_path)
      image_bytes = blob.download_as_bytes()
      image = Image.open(io.BytesIO(image_bytes)).convert('RGB')
      return image
    except Exception as e:
      print(f"Error loading image {image_path}: {str(e)}")
      return None # Return None if image loading fails

  def __getitem__(self, index):
    row = self.dataframe.iloc[index]
    miniReport = str(row['mini_report'])
    subject = str(row['subject_id'])
    study = str(row['study_id'])
    dicom = str(row['dicom_id'])
    image = self._loadImage(subject_id=subject,study_id=study,dicom_id=dicom)
    inputs = self.processor(images=image, text=self.prompt, return_tensors="pt", padding="max_length",
                            truncation=True, max_length=self.max_length)
    labels = self.processor.tokenizer(miniReport, return_tensors="pt", padding="max_length",
                                      truncation=True, max_length=self.max_length)["input_ids"]

    return {
      "pixel_values": inputs["pixel_values"],  # Shape: [1, 3, H, W]
      "input_ids": inputs["input_ids"],        # Shape: [1, max_length]
      "attention_mask": inputs["attention_mask"],  # Shape: [1, max_length]
      "labels": labels                         # Shape: [1, max_length]
    }

In [ ]:
processor = AutoProcessor.from_pretrained("microsoft/git-base")

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/ds677_ablationStudy/train_split.csv')
test_df = pd.read_csv('/content/drive/MyDrive/ds677_ablationStudy/test_split.csv')
val_df = pd.read_csv('/content/drive/MyDrive/ds677_ablationStudy/val_split.csv')

max_length = max(len(processor.tokenizer.encode(report)) for report in train_df['mini_report'])
print(f"Max length of mini-reports: {max_length}")

train_dataset = CXR(train_df, processor, max_length)
val_dataset = CXR(val_df, processor, max_length)
test_dataset = CXR(test_df, processor, max_length)

Max length of mini-reports: 92


In [ ]:
print(train_dataset[0]['pixel_values'].shape)
print(train_dataset[0]['input_ids'].shape)
print(train_dataset[0]['attention_mask'].shape)
print(train_dataset[0]['labels'].shape)

torch.Size([1, 3, 224, 224])
torch.Size([1, 92])
torch.Size([1, 92])
torch.Size([1, 92])


In [ ]:
folder_path = '/content/drive/MyDrive/ds677_ablationStudy/results'

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "mps")

In [ ]:
device

device(type='cuda')

In [ ]:
def collate_fn(batch):
    pixel_values = torch.cat([item["pixel_values"] for item in batch], dim=0)  # [batch_size, 3, H, W]
    input_ids = torch.cat([item["input_ids"] for item in batch], dim=0)        # [batch_size, max_length]
    attention_mask = torch.cat([item["attention_mask"] for item in batch], dim=0)  # [batch_size, max_length]
    labels = torch.cat([item["labels"] for item in batch], dim=0)               # [batch_size, max_length]

    #print(f"Batch shapes: pixel_values={pixel_values.shape}, input_ids={input_ids.shape}, "
    #      f"attention_mask={attention_mask.shape}, labels={labels.shape}")

    return {
        "pixel_values": pixel_values,
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [ ]:
model = AutoModelForCausalLM.from_pretrained("microsoft/git-base")

In [ ]:
for name, module in model.named_modules():
  print(name)


git
git.embeddings
git.embeddings.word_embeddings
git.embeddings.position_embeddings
git.embeddings.LayerNorm
git.embeddings.dropout
git.image_encoder
git.image_encoder.vision_model
git.image_encoder.vision_model.embeddings
git.image_encoder.vision_model.embeddings.patch_embedding
git.image_encoder.vision_model.embeddings.position_embedding
git.image_encoder.vision_model.pre_layrnorm
git.image_encoder.vision_model.encoder
git.image_encoder.vision_model.encoder.layers
git.image_encoder.vision_model.encoder.layers.0
git.image_encoder.vision_model.encoder.layers.0.self_attn
git.image_encoder.vision_model.encoder.layers.0.self_attn.k_proj
git.image_encoder.vision_model.encoder.layers.0.self_attn.v_proj
git.image_encoder.vision_model.encoder.layers.0.self_attn.q_proj
git.image_encoder.vision_model.encoder.layers.0.self_attn.out_proj
git.image_encoder.vision_model.encoder.layers.0.layer_norm1
git.image_encoder.vision_model.encoder.layers.0.mlp
git.image_encoder.vision_model.encoder.layers.0

In [ ]:
# Define target modules for LoRA
vision_modules = ["q_proj", "k_proj", "v_proj", "out_proj", "fc1", "fc2"]
text_modules = ["query", "key", "value", "dense"]

# Ablation configurations
ablation_configs = [
    {"name": "lora_vision_only", "target_modules": vision_modules},
    {"name": "lora_text_only", "target_modules": text_modules},
]

In [ ]:
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/ds677_ablationStudy/results/lora_target_modules",
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=5,
    learning_rate=1e-5,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    save_total_limit=1,  # Save only the best model
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    report_to="none",
    lr_scheduler_type="reduce_lr_on_plateau",
    optim="adamw_torch",
    label_names=["labels"]
)

bleu = load("bleu")

def evaluate_model(model, test_loader, processor, max_length, device):
    processor.tokenizer.padding_side = 'left'
    model.eval()
    test_loss = 0.0
    test_samples = 0
    predictions = []
    references = []

    with torch.no_grad():
        for batch in test_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs.loss
            batch_size_actual = batch["pixel_values"].size(0)
            test_loss += loss.item() * batch_size_actual
            test_samples += batch_size_actual

            pixel_values = batch["pixel_values"]
            input_ids = batch["input_ids"]
            generated_ids = model.generate(
              pixel_values=pixel_values,
              input_ids=input_ids,
              max_new_tokens=50,  # Generate up to 50 new tokens
              do_sample=False,    # Use greedy decoding for reproducible results
              pad_token_id=processor.tokenizer.pad_token_id
            )
            generated_texts = processor.batch_decode(generated_ids, skip_special_tokens=True)
            predictions.extend([text.replace("List pathalogical findings for this chest X-ray:", "").strip() for text in generated_texts])

            label_ids = batch["labels"]
            reference_texts = processor.batch_decode(label_ids, skip_special_tokens=True)
            references.extend(reference_texts)

    avg_test_loss = test_loss / test_samples
    bleu_score = bleu.compute(predictions=predictions, references=[[ref] for ref in references])
    return avg_test_loss, bleu_score["bleu"]

In [91]:
results = []
for config in ablation_configs:
    print(f"\nRunning ablation experiment: {config['name']}")

    model = AutoModelForCausalLM.from_pretrained("microsoft/git-base")
    processor = AutoProcessor.from_pretrained("microsoft/git-base")

    lora_config = LoraConfig(
        r=8,
        lora_alpha=16,
        target_modules=config["target_modules"],
        lora_dropout=0.1,
        bias="none",
        task_type="CAUSAL_LM"
    )
    peft_model = get_peft_model(model, lora_config)
    peft_model.to(device)
    peft_model.print_trainable_parameters()

    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, collate_fn=collate_fn)
    val_loader = DataLoader(val_dataset, batch_size=64, collate_fn=collate_fn)
    test_loader = DataLoader(test_dataset, batch_size=64, collate_fn=collate_fn)

    # Trainer with label_names
    trainer = Trainer(
        model=peft_model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        data_collator=collate_fn,
    )

    # Track memory and time
    start_time = time.time()

    # Train using Trainer
    train_result = trainer.train()

    # Extract losses from trainer logs
    train_losses = [log["loss"] for log in trainer.state.log_history if "loss" in log]
    val_losses = [log["eval_loss"] for log in trainer.state.log_history if "eval_loss" in log]

    # Memory usage (average over epochs)
    memory_usage = []
    for epoch in range(10):
        memory = torch.cuda.memory_allocated(device) / 1024**2 if device.type == "cuda" else torch.mps.current_allocated_memory() / 1024**2
        memory_usage.append(memory)

    training_time = time.time() - start_time

    # Evaluate on test set
    test_loss, bleu_score = evaluate_model(peft_model, test_loader, processor, max_length, device)

    results.append({
        "experiment": config["name"],
        "train_losses": train_losses,
        "val_losses": val_losses,
        "memory_usage_MB": memory_usage,
        "training_time_seconds": training_time,
        "test_loss": test_loss,
        "bleu_score": bleu_score,
        "trainable_parameters": peft_model.get_nb_trainable_parameters()[0]
    })

    del peft_model
    del model
    torch.cuda.empty_cache() if device.type == "cuda" else torch.mps.empty_cache()
    gc.collect()

# Save results
folder_path = '/content/drive/MyDrive/ds677_ablationStudy/results'
os.makedirs(folder_path, exist_ok=True)
results_df = pd.DataFrame(results)
results_df.to_csv(os.path.join(folder_path, 'lora_target_modules_ablation_results.csv'), index=False)
print("\nAblation Study Results (Target Modules):")
print(results_df[["experiment", "test_loss", "bleu_score", "training_time_seconds", "trainable_parameters"]])


Running ablation experiment: lora_vision_only
trainable params: 1,327,104 || all params: 177,946,170 || trainable%: 0.7458


Epoch,Training Loss,Validation Loss
1,9.105000,8.740946
2,8.681500,8.604371


Epoch,Training Loss,Validation Loss
1,9.105000,8.740946
2,8.681500,8.604371


KeyboardInterrupt: 